<a href="https://colab.research.google.com/github/rchrdgwr/AIE4/blob/main/RIchards_Copy_of_Fine_tuning_Embedding_Models_for_RAG_Assignment_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Embeddings for RAG on Specific Data

Note: This Notebook was developed within the Colab environment and copied back to the Github repository

In this notebook we are looking at fine tuning embeddings. In particular the Snowflake model. We want to fine tune the embeddings based on the language and jargon used in a particular document - the EU IA Act.

Our process includes
- Retrieving the document
- Chunking the document using the RecursiveTextSplitter
- Creating training, validation, and test sets of documents
- Creating the fine tuning dataset for each by using an LLM to generate questions from the document chunks, making sure that we associate the question with the document it was created from
- Setting up the dataset format for later use
- Making a copy of the Snowflake embedding model
- Setting up our loss functions - MultipleNegativesRankingLoss and MatryoshkaLoss
- Setting up the Evaluator - this allows us to assess the performance of the embedding model during the fine tuning
- Train the model using the training dataset
- Run a quick evaluation to see if the fine tuned model has a better hit rate (retrieving the appropriate document) than the base model using the test dataset
- Create 2 RAG chains (base embedding model, fine tuned model) to compare answers between the models to identify any differences. We also used a different chunk size.
- Use RAGAS to do a quantifiable comparison, having RAGAS generating questions and also providing the context_recall and context_precision metrics using the test dataset
- We then review and compare these metrics and make some observations


---

As we start our "fine-tuning" week, we'll start with the lowest hanging improvement one can do for RAG - which is:

Fine-tuning embeddings!

- 🤝 Breakout Room #1:
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-m`
  - Task 5: Evaluating our Retriever

- 🤝 Breakout Room #2:
  - Task 1: Vibe Checking Our LCEL RAG Chain
  - Task 2: Ragas Evaluation



#### Basic Overview of Fine-tuning Embeddings

In essence, what we want to do when we fine-tune our embedding models is very simple:

```
Move the embeddings for questions relating to a document
closer together with that document
```

We can think of fine-tuning our embedding models as follows:

1) We have some pair of text items that *should* be closer together
  - `Question`, `Document` pairs
  - EX: `Who drives the bus?`, `The bus was driven by Kyle, the Bus Driver`.

2) We use these pairs as labeled data to fine-tune our embedding model.

The process of training helps the model more accurately associate our questions with the correct documents.

#####❓ Question #1:

Describe the nuance between using Q&D pairs to train the embedding model vs. inter-document pairs/related sentences.

What caveats does this approach have? Are there any special considerations for what kind of Q's we should use?

---

**ANSWER:**

We are specifically relating *the questions* to *the documents*. This means that we are making our embedding model at the very specific task of relating potential questions to specific documents.

- Q&D - tailored for tasks where model needs to find information related to specific queries. Emphasizing relevant documents to specific questions will enhance the model's ability to find precise answers.
- Inter-Document - aim to find similarities between texts, looking for semantic comprehension. Enhances the model to recognize related content.

There are many caveats, but the main ones are:

- Your Q's should reflect the Q's of your users
- This kind of fine-tuning will (purposefully) "overfit" on your data; this is the desired result in this case.

Additional caveats:
- documents must be relevant to the question for training
- questions should be clear and unambiguous
-

My notes on this:
- Q-D pair - question and relevant document that provides answer or pertinent information. So looking at tying a question to text and work to ensure that the correct document is more likely to be retrieved for a given question. Encourages embedding model to positions questions close to their relevant documents improving retrieval.
- Inter-document pair - related documents or sentances that share thematic, topical or semantic similarities without being tied to a specific question. So looking for similarity between 2 texts. Promotes a cohesive embedding space where related text are close together based on similarity of content.

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [ ]:
!pip install -qU langchain_openai langchain_huggingface langchain_core==0.2.39 langchain langchain_community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.0 MB/s eta 0:00:00


In [ ]:
!pip install -qU faiss-cpu unstructured==0.15.7 python-pptx==1.0.2 nltk==3.9.1

### Provide OpenAI API Key

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


## Task 2: Loading Data

We'll be using a recent document released by the EU 'laying down harmonised rules on artificial intelligence and amending Regulations'.

The data can be found [here](https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A32024R1689), though we will be using a HTML version which was collected into the AIM DataRepository.

First, we'll clone and then `cd` into the DataRepository.

In [ ]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 90 (delta 24), reused 29 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (90/90), 70.26 MiB | 48.03 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
%cd DataRepository

/content/DataRepository/DataRepository/DataRepository


Next we're going to be using the `UnstructuredHTMLLoader` to load our HTML document into a LangChain document using the [Unstructured](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.html.UnstructuredHTMLLoader.html) library.

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

training_documents_loaded = UnstructuredHTMLLoader("eu_ai_act.html")

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

Next we can load/split these documents as follows.

In [ ]:
training_documents = text_splitter.split_documents(training_documents_loaded.load())

In [ ]:
len(training_documents)

1029

Next, we're going to associate each of our chunks with a unique identifier.

In [ ]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [ ]:
training_split_documents = training_documents[:300]
validation_split_documents = training_documents[300:350]
test_split_documents = training_documents[350:400]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [July 18th](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [ ]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [ ]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

##### 🏗️ Activity #1:

We have:

- Lists of `Documents` with the `metadata` field `id`.

We need:

- An object with key `id`, which have values `str` questions.
- An object with key `question_id`, which have values `List(str)` which will be a list of associated `context_id`.

An Example:

question_object:
```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': 'What types of accessible formats are available for persons with disabilities?',
'df58ee4f-714c-419e-8324-94e5870574e2': 'How do accessible formats benefit persons with disabilities?',
'505fce8b-0e56-48de-a251-61027e396918': 'What are some of the risks associated with the increasing capabilities of AI systems that generate synthetic content?',
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': 'Why is it important for providers of AI systems to embed technical solutions for marking and detecting synthetic content?'
}
 ```

 context_object:
 ```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'df58ee4f-714c-419e-8324-94e5870574e2': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'505fce8b-0e56-48de-a251-61027e396918': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
}
 ```

 As you can see, a piece of context can be associated with more than 1 question.

 The task is to write the Python function(s) to accomplish this task.

 Your function signature is provided below, along with the desired return values.

 > NOTE: You can make any modifications that you desire - assuming that you have the correct input and outputs.

In [ ]:
import tqdm

def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  for document in tqdm.tqdm(documents):
    questions_created = question_generation_chain.invoke({
        "context" : document.page_content,
        "n_questions" : n_questions
    })
    # print(questions_created)
    for question in questions_created.content.split("\n"):
      if question:
        question_id = str(uuid.uuid4())
        parts = question.split('. ', 1)
        if len(parts) > 1:
            question_no_number = parts[1]
        else:
            question_no_number = question
        # print(question_no_number.strip())
        questions[question_id] = question_no_number.strip()
        relevant_docs[question_id] = [document.metadata["id"]]

  return questions, relevant_docs

We'll use the function to generate training, validation, and test data with `n_questions=2` for each.

Lets run the tests to ensure the format that is being returned

content='1. What is the main purpose of Regulation (EU) 2024/1689 as outlined in the context?  \n2. Which specific regulations and directives are amended by the Artificial Intelligence Act mentioned in the context?'

In [ ]:
# trial_documents = training_documents[:1]
# create_questions(trial_documents, 2)

In [ ]:
num_questions = 2
training_questions, training_relevant_contexts = create_questions(training_split_documents, num_questions)

100%|██████████| 300/300 [05:35<00:00,  1.12s/it]


In [ ]:
validation_questions, validation_relevant_contexts = create_questions(validation_split_documents, 2)

100%|██████████| 50/50 [00:34<00:00,  1.46it/s]


In [ ]:
test_questions, test_relevant_contexts = create_questions(test_split_documents, 2)

100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

> NOTE: If you ran into issues creating the data - you can use the data from the DataRespository. It's simply called: `train_dataset.jsonl`, etc.

In [ ]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [ ]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in validation_split_documents}

val_dataset = {
    "questions" : validation_questions,
    "relevant_contexts" : validation_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [ ]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-m`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-m`](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

In [ ]:
!pip uninstall -y pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0


In [ ]:
!pip install -qU sentence_transformers datasets pyarrow==14.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 14.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.3.1 which is incompatible.


In [ ]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

> NOTE: It is typical to use a much larger batch size (~64+), hardware permitting.

In [ ]:
BATCH_SIZE = 20

Let's move our dataset into the expected format for training.

In [ ]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [ ]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

Loss is an important part of training, fine-tuning, and more. If you want a deep dive on loss - you can check out our [event on loss!](https://www.youtube.com/watch?v=iB8FWR9aD5Q&t=8s).

The core loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

This is "wrapped" in `MatryoshkaLoss`, which you can read the implementation of [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

##### 🏗️ Activity #2:

Both of these losses sound "cool", but what are they - exactly - under the hood?

Why are these losses specifically doing? Please write a short summary of each loss.

> NOTE: This is a course focused on AI Engineering and the application of AI - looking for a hint? Try pasting the code (linked above) into ChatGPT/Claude to write the summary!

Both the MultipleNegativesRankingLoss and the MatryoshkaLoss are specialized loss functions designed to improve the training of embedding models.

The MultipleNegativesRankingLoss is used for training embedding models typically where there is a "positive" pair (in this case a question and the correct document) together with multiple negative examples (all the other pairs) within the same batch. The goal is to push the embeddings of positive pairs closer together while pushing the embeddings of negative pairs further apart. The loss aims to maximise the similarity of the positive pairs while minimizing the similarity of negative pairs.

The MatryoshkaLoss builds upon a base loss function - in our case the MultipleNegativesRankingLoss - to train multiple layers of embeddings simultaneously. It trains the model to produce embeddings at multiple dimensions simultaneously. This allows the model to capture semantic information at various granularities. The Matryoshka Dimensions are a list of embeddings that define the hierarchy or layers of embeddings to be trained. The model prodcues embeddings at each specified dimension. These capture different levels of abstraction. Each layer is trained using the base loss function which ensures that higher dimensional embeddings capture more detailed semantics. MatryoshkaLoss runs the training of the specified embedding layers using the provided inner_train_loss. It then aggregates the losses from each layer using weighting schemes to balance their contrinutions.

Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [ ]:
EPOCHS = 5

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

Typically warmup steps are 10% of the epochs

In [ ]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50,
)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100,Dot Accuracy@1,Dot Accuracy@3,Dot Accuracy@5,Dot Accuracy@10,Dot Precision@1,Dot Precision@3,Dot Precision@5,Dot Precision@10,Dot Recall@1,Dot Recall@3,Dot Recall@5,Dot Recall@10,Dot Ndcg@10,Dot Mrr@10,Dot Map@100
30,No log,No log,0.870000,0.960000,0.990000,1.000000,0.870000,0.320000,0.198000,0.100000,0.870000,0.960000,0.990000,1.000000,0.937362,0.916762,0.916762,0.870000,0.960000,0.990000,1.000000,0.870000,0.320000,0.198000,0.100000,0.870000,0.960000,0.990000,1.000000,0.937362,0.916762,0.916762
50,No log,No log,0.880000,0.960000,1.000000,1.000000,0.880000,0.320000,0.200000,0.100000,0.880000,0.960000,1.000000,1.000000,0.944645,0.926167,0.926167,0.880000,0.960000,1.000000,1.000000,0.880000,0.320000,0.200000,0.100000,0.880000,0.960000,1.000000,1.000000,0.944645,0.926167,0.926167
60,No log,No log,0.860000,0.960000,1.000000,1.000000,0.860000,0.320000,0.200000,0.100000,0.860000,0.960000,1.000000,1.000000,0.937263,0.916167,0.916167,0.860000,0.960000,1.000000,1.000000,0.860000,0.320000,0.200000,0.100000,0.860000,0.960000,1.000000,1.000000,0.937263,0.916167,0.916167
90,No log,No log,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.934390,0.912500,0.912500,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.934390,0.912500,0.912500
100,No log,No log,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.933513,0.911500,0.911500,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.933513,0.911500,0.911500
120,No log,No log,0.850000,0.950000,1.000000,1.000000,0.850000,0.316667,0.200000,0.100000,0.850000,0.950000,1.000000,1.000000,0.932441,0.909833,0.909833,0.850000,0.950000,1.000000,1.000000,0.850000,0.316667,0.200000,0.100000,0.850000,0.950000,1.000000,1.000000,0.932441,0.909833,0.909833
150,No log,No log,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.936132,0.914833,0.914833,0.860000,0.950000,1.000000,1.000000,0.860000,0.316667,0.200000,0.100000,0.860000,0.950000,1.000000,1.000000,0.936132,0.914833,0.914833


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [ ]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [ ]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
# purpose of this is to see if the expected document was successfully retrieved
# get all the documents, questions, and relevant contexts
# set up Document objects for each page in the corpus and include the content and document id
# add the documents to a FAISS vector store and create the retriever
# get the k documents for each question and find their ids
# check if the expected id is in this list - set is_hit if it is
# create a list of dictionaries for each question
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-m`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [ ]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 100/100 [00:14<00:00,  6.83it/s]


In [ ]:
te3_results_df = pd.DataFrame(te3_results)

In [ ]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

0.98

### `Snowflake/snowflake-arctic-embed-m` (base)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-m")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 100/100 [00:01<00:00, 84.00it/s]


In [ ]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [ ]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.58

### `Snowflake/snowflake-arctic-embed-m` (fine-tuned)

In [ ]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 100/100 [00:01<00:00, 85.21it/s]


In [ ]:
finetune_results_df = pd.DataFrame(finetune_results)

In [ ]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

1.0

# 🤝 Breakout Room #2

## Task 1: Vibe Checking the Embedding Models using the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(training_documents_loaded.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [ ]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [ ]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
base_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to promote a human-centric approach to AI, ensure the development of secure, trustworthy, and ethical AI, protect ethical principles, and facilitate the protection of natural persons, undertakings, democracy, the rule of law, and environmental protection. Additionally, the regulation aims to boost innovation and employment, positioning the Union as a leader in the uptake of trustworthy AI.'

In [ ]:
base_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'I do not know.'

In [ ]:
base_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and competence in performing a wide range of tasks. Therefore, it can be inferred that having a billion parameters is a threshold for being considered a model with "too many" parameters in this context.'

In [ ]:
base_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is a type of artificial intelligence (AI) technology designed to identify and interpret human emotions based on various inputs, such as facial expressions, voice tone, body language, or biometric data. These systems analyze patterns in the data to infer the emotional state of an individual.\n\nThe importance of emotion recognition systems lies in their potential applications across various fields, including mental health, customer service, security, and human-computer interaction. They can enhance user experiences, improve communication, and provide insights into emotional well-being. However, there are significant concerns regarding their reliability, specificity, and potential for discriminatory outcomes, particularly given the variability of emotional expression across different cultures and individuals.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [ ]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [ ]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [ ]:
finetune_rag_chain.invoke({"question" : "Why does the EU want to regulate AI?"})["response"]

'The EU wants to regulate AI to improve the functioning of the internal market, promote the uptake of human-centric and trustworthy artificial intelligence, and ensure a high level of protection of health, safety, and fundamental rights as enshrined in the Charter of Fundamental Rights of the European Union. Additionally, the regulation aims to protect against the harmful effects of AI systems, support innovation, and ensure the free movement of AI-based goods and services across Member States.'

In [ ]:
finetune_rag_chain.invoke({"question" : "What are the codes of practice?"})["response"]

'Codes of practice are guidelines developed to ensure compliance with regulations concerning the use of general-purpose AI models, particularly those that present systemic risks. They are intended to cover obligations for providers of these AI models and help establish a risk taxonomy at the Union level, including identifying sources of systemic risks. The AI Office is responsible for encouraging and facilitating the creation, review, and adaptation of these codes, which should reflect the state of the art and consider diverse perspectives. Providers of general-purpose AI models can rely on these codes to demonstrate compliance with their obligations under the regulation. The codes of practice are expected to be ready by May 2, 2025.'

In [ ]:
finetune_rag_chain.invoke({"question" : "How many parameters is too many parameters?"})["response"]

'The context suggests that models with at least a billion parameters are considered to display significant generality and to competently perform a wide range of distinctive tasks. Therefore, it can be inferred that having at least a billion parameters is seen as a threshold for significant generality, but there is no specific limit stated for "too many parameters."'

In [ ]:
finetune_rag_chain.invoke({"question" : "What is an emotion recognition system and why is it important?"})["response"]

'An emotion recognition system is defined as an AI system that identifies or infers the emotions or intentions of natural persons based on their biometric data. This includes emotions such as happiness, sadness, anger, surprise, disgust, embarrassment, excitement, shame, contempt, satisfaction, and amusement. It is important because it can have significant implications for various applications, including security, marketing, and mental health. However, there are serious concerns regarding the scientific basis of these systems, as emotions can vary widely across cultures and individuals, leading to potential discriminatory outcomes and intrusions on personal rights and freedoms.'

#####❓Question #2:

Which LCEL RAG Chain do you think answered the questions better, and why?

The answers by both chains were very similar except the base Snowflake model failed to answer one of the questions (response of "I don't know"). So based on this the fine-tuned model responded somewhat better.

Since the fine-tuned model had been trained on questions created from the documents, this allowed the embeddings for the question and relevant document to be moved closer resulting in a better chance for the correct document to be retrieved for questions similar to those that the embedding model was trained on.

## Task 2: RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

In [ ]:
!pip install -qU ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.7 MB/s eta 0:00:00


### RAGAS Synthetic Testset Generation

First things first, we need to generate some data to test our model on.

Let's use our test data that we created before as a base!

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

In [ ]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

In [ ]:
print(test_split_documents)

[Document(metadata={'source': 'eu_ai_act.html', 'id': 'c3246fe6-52bd-41c0-9b2e-1ce0f811854f'}, page_content='(154) The national competent authorities should exercise their powers independently, impartially and without bias, so as to safeguard the principles of objectivity of their activities and tasks and to ensure the application and implementation of this Regulation. The members of these authorities should refrain from any action incompatible with their duties and should be subject to confidentiality rules under this Regulation.'), Document(metadata={'source': 'eu_ai_act.html', 'id': '2bf175ad-3d24-4525-92bc-70e427bfc28c'}, page_content='(155) In order to ensure that providers of high-risk AI systems can take into account the experience on the use of high-risk AI systems for improving their systems and the design and development process or can take any possible corrective action in a\xa0timely manner, all providers should have a\xa0post-market monitoring system in place. Where releva

In [ ]:
testset = generator.generate_with_langchain_docs(test_split_documents, test_size=20, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25}, raise_exceptions=False)

In [ ]:
testset.to_pandas().head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does Regulation (EU) 2019/1020 play ...,[(161) It is necessary to clarify the responsi...,Regulation (EU) 2019/1020 plays a role in the ...,simple,"[{'source': 'eu_ai_act.html', 'id': '3a3bec52-...",True
1,How should voluntary codes of conduct be devel...,[To ensure that the voluntary codes of conduct...,Voluntary codes of conduct should be based on ...,simple,"[{'source': 'eu_ai_act.html', 'id': '0ea0f8cc-...",True
2,What role do competent authorities play in sup...,[( 50 ) and (EU) 2016/97 ( 51 ) of the Europea...,Competent authorities play a crucial role in s...,simple,"[{'source': 'eu_ai_act.html', 'id': 'a0c03c17-...",True
3,What role does the European Parliament play in...,[( 50 ) and (EU) 2016/97 ( 51 ) of the Europea...,The European Parliament does not play a direct...,simple,"[{'source': 'eu_ai_act.html', 'id': 'a0c03c17-...",True
4,How do data protection authorities play a role...,[(157) This Regulation is without prejudice to...,Data protection authorities play a crucial rol...,simple,"[{'source': 'eu_ai_act.html', 'id': '77dc5035-...",True


### Generating Answer Datasets

For each of our pipelines, let's generate answers to these questions!

Once we have our: Questions, Answers, Contexts, Ground Truths we can move on to evaluating our datasets!

In [ ]:
from datasets import Dataset

def generate_answers(chain, testset):
  answers = []
  contexts = []
  questions = testset.to_pandas()["question"].values.tolist()
  ground_truths = testset.to_pandas()["ground_truth"].values.tolist()

  for question in tqdm.tqdm(questions):
    answer = chain.invoke({"question" : question})
    answers.append(answer["response"])
    contexts.append([context.page_content for context in answer["context"]])

  return Dataset.from_dict({
      "question" : questions,
      "answer" : answers,
      "contexts" : contexts,
      "ground_truth" : ground_truths
  })

In [ ]:
base_dataset = generate_answers(base_rag_chain, testset)

100%|██████████| 18/18 [00:14<00:00,  1.22it/s]


In [ ]:
finetune_dataset = generate_answers(finetune_rag_chain, testset)

100%|██████████| 18/18 [00:22<00:00,  1.24s/it]


### Evaluating Using the Test Set

Now that we have a test set - it's time to evaluate our pipelines with it!

In [ ]:
from ragas.metrics import (
    context_recall,
    context_precision,
)

In [ ]:
from ragas import evaluate

result = evaluate(
    base_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

In [ ]:
result

{'context_precision': 0.6386, 'context_recall': 0.4361}

In [ ]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What role does Regulation (EU) 2019/1020 play ...,[of the high-risk AI system with the requireme...,I do not know.,Regulation (EU) 2019/1020 plays a role in the ...,0.466667,0.000000
1,How should voluntary codes of conduct be devel...,"[three years thereafter, the Commission should...",Voluntary codes of conduct should be developed...,Voluntary codes of conduct should be based on ...,0.887500,0.333333
2,What role do competent authorities play in sup...,[as defined in Regulation (EU) No 575/2013 of ...,Competent authorities are designated to superv...,Competent authorities play a crucial role in s...,1.000000,0.666667
3,What role does the European Parliament play in...,[as defined in Regulation (EU) No 575/2013 of ...,"The European Parliament, along with the Counci...",The European Parliament does not play a direct...,1.000000,1.000000
4,How do data protection authorities play a role...,[equipment. Those Union legal acts provide the...,Data protection authorities play a crucial rol...,Data protection authorities play a crucial rol...,1.000000,0.500000


In [ ]:
result = evaluate(
    finetune_dataset,
    metrics=[
        context_precision,
        context_recall,
    ],
)

In [ ]:
result

{'context_precision': 0.8969, 'context_recall': 0.8648}

In [ ]:
result.to_pandas().head()

,question,contexts,answer,ground_truth,context_precision,context_recall
0,What role does Regulation (EU) 2019/1020 play ...,[1. Where an AI system is based on a general-p...,Regulation (EU) 2019/1020 provides the framewo...,Regulation (EU) 2019/1020 plays a role in the ...,1.0,1.000000
1,How should voluntary codes of conduct be devel...,"[with disability, stakeholders’ participation ...",Voluntary codes of conduct should be developed...,Voluntary codes of conduct should be based on ...,1.0,0.666667
2,What role do competent authorities play in sup...,[as defined in Regulation (EU) No 575/2013 of ...,Competent authorities are designated to superv...,Competent authorities play a crucial role in s...,1.0,1.000000
3,What role does the European Parliament play in...,[as defined in Regulation (EU) No 575/2013 of ...,The European Parliament plays a role in establ...,The European Parliament does not play a direct...,1.0,1.000000
4,How do data protection authorities play a role...,[(157) This Regulation is without prejudice to...,Data protection authorities play a role in sup...,Data protection authorities play a crucial rol...,1.0,1.000000


#### 🏗️ Activity #3:

Discuss changes that you'd make to this pipeline based on the performance improvements that you see with RAGAS and the fine-tuning.

Come up with 3 changes, and then we'll discuss these options as a group!

Lets look at the overall results first:
- Base embedding model: {'context_precision': 0.6386, 'context_recall': 0.4361}
- Fine-tuned model: {'context_precision': 0.8969, 'context_recall': 0.8648}

The fine tuned model has shown significant improvement in both context precision and context recall
- Context precision - accuracy of retrieved documents in terms of relevance to the question - higher means more retrieved documents are relevant and less irrelevant information
- Context recall - completeness of the retrieval process ie proportion of relevant documents returned - higher means retrieval of more of the relevant documents, giving better coverage

Some things that could be changed in the pipeline


1. Continue to fine tune the embedding model by increasing the documents the model is trained on (ie add to the document corpus adding more varied sources and formats) and using more questions
2. Add keyword-based methods in addition to the embedding-based retrieval to improve the accuracy of document retrieval
3. Use some of the techniques previously employed to improve the retrieval process such as Contextual Compression (Reranker), MultiQueryRetriever and/or Parent Document Retriever